In [1]:
import socket
import os
from _thread import *
from random import randrange, getrandbits
from math import gcd

### Generating big prime numbers using Miller Rabin Test
<a href='https://medium.com/@ntnprdhmm/how-to-generate-big-prime-numbers-miller-rabin-49e6e6af32fb'>Source</a>

In [5]:
def is_prime(n, k=128):
    """ Test if a number is prime
        Args:
            n -- int -- the number to test
            k -- int -- the number of tests to do
        return True if n is prime
    """
    # Test if n is not even.
    # But care, 2 is prime !
    if n == 2 or n == 3:
        return True
    if n <= 1 or n % 2 == 0:
        return False
    # find r and s
    s = 0
    r = n - 1
    while r & 1 == 0:
        s += 1
        r //= 2
    # do k tests
    for _ in range(k):
        a = randrange(2, n - 1)
        x = pow(a, r, n)
        if x != 1 and x != n - 1:
            j = 1
            while j < s and x != n - 1:
                x = pow(x, 2, n)
                if x == 1:
                    return False
                j += 1
            if x != n - 1:
                return False
    return True

def generate_prime_candidate(length):
    """ Generate an odd integer randomly
        Args:
            length -- int -- the length of the number to generate, in bits
        return a integer
    """
    # generate random bits
    p = getrandbits(length)
    # apply a mask to set MSB and LSB to 1
    p |= (1 << length - 1) | 1
    return p
def generate_prime_number(length=1024):
    """ Generate a prime
        Args:
            length -- int -- length of the prime to generate, in          bits
        return a prime
    """
    p = 4
    # keep generating while the primality test fail
    while not is_prime(p, 128):
        p = generate_prime_candidate(length)
    return p


### Key generation

In [ ]:
def generate_public_key(phi):
    e = phi-1
    while True:
        if gcd(e, phi) == 1:
            return e
        e -= 1

In [6]:
def generate_key():
    p = generate_prime_number()
    q = generate_prime_number()
    n = p * q
    phi = (p-1) * (q-1)
    e = generate_public_key(phi)
    d = pow(e, -1, phi)
    return (e, n), (d, n)

### Encryption

In [7]:
def encrypt(pk, plaintext):
    # Unpack the key into it's components
    key, n = pk
    # Convert each letter in the plaintext to numbers based on the character using a^b mod m
    cipher = [pow(ord(char), key, n) for char in plaintext]
    # Return the array of bytes
    return cipher

### Decryption

In [8]:
def decrypt(pk, ciphertext):
    # Unpack the key into its components
    key, n = pk
    # Generate the plaintext based on the ciphertext and key using a^b mod m
    plain = [chr(pow(char, key, n)) for char in ciphertext]
    # Return the array of bytes as a string
    return ''.join(plain)

In [10]:

public_key, private_key = generate_key()
print(public_key, private_key)

(65537, 31326754858867358124357086579071826010200059323531955679566887945759390675838673845232015105126972937388568736544596689797668825339157136890481595384743810006624619157134135391484571402535617730110112660311598998153160586974262941255564293668481979962157625919021316451251911741026695391775636810992585738524913593376057045111861800302051912786822232202770027007361078797914806407627383850375687974820789943593511606243328071372907407506615220970113618164339212696880056111445944606126460979705361190756332606801944055865963930780537469800597900982892354281404585770500536757133376053019637166714789307054013578200747) (497264493029581956097604058290865016683875089098834147023108069181889529885026357648272965101600469151247815075870789575301202053806630903263316961086851481604156298108955872984137199292884982904825521469126698472294016488843336402157003443906828261205617868433981956821907682441827841006823549666228038265210481484995650996118396575931853761650380128702132064134431185425

In [ ]:
# print('Ciphertext: ', encrypt(pk, 'The quick brown fox jumps over the lazy dog.'))
# print('Plaintext: ', decrypt(sk, encrypt(pk, 'The quick brown fox jumps over the lazy dog.')))

In [ ]:
ServerSideSocket = socket.socket()
host = '127.0.0.1'
port = 2004
ThreadCount = 0
try:
    ServerSideSocket.bind((host, port))
except socket.error as e:
    print(str(e))
print('Socket is listening..')
ServerSideSocket.listen(5)
def multi_threaded_client(connection):
    connection.send(str.encode('Server is working:'))
    while True:
        data = connection.recv(2048)
        print('Data received from the client {}'.format(data.decode('utf-8')))
        response = 'Server message: ' + data.decode('utf-8')
        if not data:
            break
        connection.sendall(str.encode(response))
    connection.close()
while True:
    Client, address = ServerSideSocket.accept()
    print('Connected to: ' + address[0] + ':' + str(address[1]))
    start_new_thread(multi_threaded_client, (Client, ))
    ThreadCount += 1
    print('Thread Number: ' + str(ThreadCount))
ServerSideSocket.close()

Socket is listening..
Connected to: 127.0.0.1:53587
Thread Number: 1
Data received from the client yo wassup
Data received from the client hello world
